In [1]:
import pandas as pd
import numpy as np 
import random as rd

In [2]:
D = pd.read_json (r'disease_db.json')
# put the json into the same direct where this file lies 

In [3]:
D
# D is the Pandas Dataframe 

,name,id,symptoms
0,Cold,80,"[{'id': 28, 'probability': 0.7058823390000001}..."
1,Flu,11,"[{'id': 54, 'probability': 0.54}, {'id': 29, '..."
2,Hay fever,367,"[{'id': 28, 'probability': 0.8823529000000001}..."
3,State of stress,606,"[{'id': 54, 'probability': 0.9}, {'id': 131, '..."
4,Depression,47,"[{'id': 238, 'probability': 0.72}, {'id': 54, ..."
5,Flu-related rhinitis,368,"[{'id': 11, 'probability': -1.0}, {'id': 15, '..."
6,Inflammation of paranasal sinuses,41,"[{'id': 64, 'probability': 0.48959995300000003..."
7,Lack of supply of thyroid hormones,60,"[{'id': 54, 'probability': 0.72}, {'id': 123, ..."
8,Sleeping disorder,379,"[{'id': 16, 'probability': 0.9}, {'id': 52, 'p..."
9,Excessive feeling of fear,181,"[{'id': 238, 'probability': 0.9}, {'id': 17, '..."


In [4]:
# Function that parses one Input str into an array 
# gets Symptom row -> return array has form [id1,acc1,id2,acc2...]
def get_att_for_line(d):
    d = d.split(',')
    # 1,20 because of 10 Symptom cap ? higher this number if more symptomslots are needed
    matTemp = np.zeros((1,100))
    for n,i in enumerate(d):
        f = i.partition(':')[2]
        f = f.partition('}')[0]
        matTemp[0][n] = f
    
    return matTemp

In [5]:
# returns a attribute matrix for Symptoms:
# returns a matrix like :[[id1(disease1),acc1(disease1),id2(disease1)...],
#                         [id1(disease2),acc1(diesease2),id2(disease1)..], 
#                         ...]
def get_att_matrix():
    mat = np.zeros((D.shape[0],100))
    for index, row in D.iterrows():
        rowstr = get_att_for_line(str(row['symptoms']))
        for i in range(rowstr.shape[1]):
            mat[index][i] = rowstr[0][i]
    #print(mat)
    return mat

In [6]:
mat = get_att_matrix()
# every row == 1 disease every column one Symptom ID
# IDS is a matrix looking like this: [[id1(disease1), id2(disease1), id3(disease1),...],
#                                     [id1(disease2), id2(disease2), id3(disease2),...],
#                                   ,..,]
IDS = mat[::,0::2]
# every row == 1 disease every column one Acc %
# SYM is a matrix looking like this: [[acc1(disease1), acc2(disease1), acc3(disease1),...],
#                                     [acc1(disease2), acc2(disease2), acc3(disease2),...],
#                                   ,..,]
SYM = mat[::,1::2]

In [7]:
def get_symptoms_and_acc_cut(ids,sym,row):
    # get the indicies of nonNull ID's aka all valid ID's and return the cut arrays 
    IDSS = ids[row].nonzero()
    # get the count of valid Symptoms
    length = len(IDSS[0])
    # cut the array to req length
    IDST = ids[row,0:length:]
    SYMT = sym[row,0:length:]
    # del -1 in ids
    neg = np.argwhere(IDST < 0).flatten()
    IDST = np.delete(IDST,neg)
    SYMT = np.delete(SYMT,neg)
    # del -1 in acc
    # neg = np.argwhere(SYMT == -1).flatten()
    # IDST = np.delete(IDST,neg)
    # SYMT = np.delete(SYMT,neg)
    return IDST,SYMT

In [8]:
''' SYMPTOMS APPLICATION'''

def get_symptoms_used(IDS,SYM):
    IDST,SYMT =  get_symptoms_and_acc_cut(IDS,SYM,0)
    for x in range(1,mat.shape[0]):
        IDSTt,SYMTt =  get_symptoms_and_acc_cut(IDS,SYM,x)
        IDST = np.concatenate((IDST, IDSTt), axis=0)
    IDST =np.unique(IDST,axis=0)
    return IDST

In [9]:
symliste = get_symptoms_used(IDS,SYM)
flist = []
for sss in symliste:
    flist.append(int(sss))
df=pd.read_json('Symptoms/Symptoms.json')
df.to_csv('Symptoms.csv')
patientData_reviews = pd.read_csv('Symptoms.csv')
df=pd.read_csv('Symptoms.csv', index_col=[0])
boolean_series = df.ID.isin(flist)
filtered_df = df[boolean_series]
filtered_df.to_csv('Symptoms.csv',index=None)
symidvec = np.asarray(flist)

In [10]:
def generate_patients(Label,IDST,SYMT,count):
    vec = np.full(count,int(Label))
    for n,i in enumerate(symidvec):
        argdes = np.argwhere(IDST == i).flatten()
        if len(argdes )== 0:
            vec = np.vstack((vec, np.zeros(count,)))
        else:
            ii = argdes[0]
            if SYMT[ii] < 0:
                rz = rd.uniform(0, 0.5)
                vec = np.vstack((vec,rd.choices([1,0], weights=[rz,1-rz],k=count)))
            else:
                vec = np.vstack((vec,rd.choices([1,0], weights=[SYMT[ii],1-SYMT[ii]],k=count)))
    return vec.transpose()

In [11]:
def generate_database_mat(nr):
    IDST,SYMT =  get_symptoms_and_acc_cut(IDS,SYM,0)
    Patients = generate_patients(D['id'][0],IDST,SYMT,nr)
    print(Patients.shape)
    for x in range(1,mat.shape[0]):
        IDST,SYMT =  get_symptoms_and_acc_cut(IDS,SYM,x)
        Patients = np.vstack((Patients, generate_patients(D['id'][x],IDST,SYMT,nr)))
        #ListPatients.append(generate_patients(D['id'][x],IDST,SYMT,nr) )
    return Patients

In [12]:
def get_frames(nr):
    head = np.append([0],symidvec).astype(int)
    ListPatientss = generate_database_mat(nr)
    ListPatientss = np.vstack(( np.append([0],symidvec).astype(int), ListPatientss))
    print(ListPatientss.shape)
    DIS = pd.DataFrame(data=ListPatientss[1:,:],columns=ListPatientss[0,:])
        
    return DIS

In [14]:
kk = get_frames(100)
kk.to_csv("./Patients/patients{}.csv".format("Patients"), index=False, encoding='utf8')
#data.to_csv("output_excel_file.xlsx", sheet_name="Sheet 1", index=False)

    

(100, 118)
(2001, 118)
